## Importações e Configurações

In [9]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.io as pio
import time

# Gráficos interativos na célula
pio.renderers.default = "notebook_connected"

# %%
BASE_URL = "https://www.nic.br"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}


## Scraper

In [ ]:
def montar_url(pagina):
    return f"https://www.nic.br/noticias/indice/?page={pagina}"


In [10]:
# %%
header_url = f"{BASE_URL}/noticias/indice/header.json"
header = requests.get(header_url, headers=HEADERS).json()

pd.DataFrame(header)


,codigo,titulo_pt_br,quantidade_total,totalPorArquivo
0,eventos,Eventos,1,500
1,na-midia,Na Mídia,19533,500
2,notas,Notas,231,500
3,releases,Releases,864,500
4,todos,Todos,1141,500


In [12]:
# %%
def carregar_indice_json(codigo="todos", numero_arquivo=1):
    url = f"{BASE_URL}/noticias/indice/{codigo}/indice-{numero_arquivo}.json"
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    return r.json()


In [13]:
# %%
noticias = []

PAGINAS = 5
POR_PAGINA = 10
TOTAL = PAGINAS * POR_PAGINA

dados = carregar_indice_json("todos", 1)

for noticia in dados[:TOTAL]:
    noticias.append({
        "titulo": noticia.get("titulo"),
        "link": noticia.get("permalink"),
        "data": noticia.get("data"),
        "resumo": noticia.get("resumo"),
        "categoria": noticia.get("categoria", "todos")
    })

df_nic = pd.DataFrame(noticias)
print(f"Total coletado: {len(df_nic)}")
df_nic.head()


Total coletado: 50


,titulo,link,data,resumo,categoria
0,'Influprofissionais': linguagem das redes rede...,/noticia/na-midia/influprofissionais-linguagem...,2025-12-12,"Divulgada nesta semana, a pesquisa TIC Domic&i...",na-midia
1,"<i>Cyberbullying</i>, como se precaver e procu...",/noticia/na-midia/i-cyberbullying-i-como-se-pr...,2025-12-12,<em>Mais de um ter&ccedil;o dos jovens entre 1...,na-midia
2,A relação simbiótica entre jornalismo e educaç...,/noticia/na-midia/a-relacao-simbiotica-entre-j...,2025-12-12,O Pr&ecirc;mio ANJ de Liberdade de Imprensa de...,na-midia
3,Para além dos bloqueios: queremos mudanças no...,/noticia/na-midia/para-alem-dos-bloqueios-quer...,2025-12-11,<em>A camada de transporte n&atilde;o deve dec...,na-midia
4,15ª Semana da Infraestrutura da Internet no Br...,/noticia/na-midia/15-semana-da-infraestrutura-...,2025-12-11,<em>Evento acontece entre os dias 15 e 19 de d...,na-midia


## Filtro + Gráficos

In [ ]:

df_nic["data"] = pd.to_datetime(df_nic["data"], errors="coerce")
df_nic.sort_values("data", ascending=False, inplace=True)
df_nic.reset_index(drop=True, inplace=True)

df_nic.head()


,titulo,link,data,resumo,categoria
0,'Influprofissionais': linguagem das redes rede...,/noticia/na-midia/influprofissionais-linguagem...,2025-12-12,"Divulgada nesta semana, a pesquisa TIC Domic&i...",na-midia
1,A relação simbiótica entre jornalismo e educaç...,/noticia/na-midia/a-relacao-simbiotica-entre-j...,2025-12-12,O Pr&ecirc;mio ANJ de Liberdade de Imprensa de...,na-midia
2,"<i>Cyberbullying</i>, como se precaver e procu...",/noticia/na-midia/i-cyberbullying-i-como-se-pr...,2025-12-12,<em>Mais de um ter&ccedil;o dos jovens entre 1...,na-midia
3,Para além dos bloqueios: queremos mudanças no...,/noticia/na-midia/para-alem-dos-bloqueios-quer...,2025-12-11,<em>A camada de transporte n&atilde;o deve dec...,na-midia
4,"Evento debate segurança, automação e os rumos ...",/noticia/na-midia/15-semana-de-infraestrutura-...,2025-12-11,"Entre os dias 15 e 19 deste m&ecirc;s, em S&at...",na-midia


In [15]:

keywords = ["internet", "governança", "segurança", "privacidade", "IA", "dados"]
pattern = "|".join(keywords)

df_filt = df_nic[
    df_nic["titulo"].str.contains(pattern, case=False, na=False) |
    df_nic["resumo"].str.contains(pattern, case=False, na=False)
]

print(f"{len(df_filt)} notícias filtradas")
df_filt.head()


50 notícias filtradas


,titulo,link,data,resumo,categoria
0,'Influprofissionais': linguagem das redes rede...,/noticia/na-midia/influprofissionais-linguagem...,2025-12-12,"Divulgada nesta semana, a pesquisa TIC Domic&i...",na-midia
1,A relação simbiótica entre jornalismo e educaç...,/noticia/na-midia/a-relacao-simbiotica-entre-j...,2025-12-12,O Pr&ecirc;mio ANJ de Liberdade de Imprensa de...,na-midia
2,"<i>Cyberbullying</i>, como se precaver e procu...",/noticia/na-midia/i-cyberbullying-i-como-se-pr...,2025-12-12,<em>Mais de um ter&ccedil;o dos jovens entre 1...,na-midia
3,Para além dos bloqueios: queremos mudanças no...,/noticia/na-midia/para-alem-dos-bloqueios-quer...,2025-12-11,<em>A camada de transporte n&atilde;o deve dec...,na-midia
4,"Evento debate segurança, automação e os rumos ...",/noticia/na-midia/15-semana-de-infraestrutura-...,2025-12-11,"Entre os dias 15 e 19 deste m&ecirc;s, em S&at...",na-midia


In [16]:
# %%
df_plot = df_nic.copy()

df_plot["titulo"] = df_plot["titulo"].fillna("").astype(str)
df_plot["resumo"] = df_plot["resumo"].fillna("").astype(str)
df_plot["categoria"] = df_plot["categoria"].fillna("Indefinida")


In [17]:
# %%
def plot_charts(df):
    if df.empty:
        print("Sem dados para gráficos.")
        return

    # =========================
    # 1️⃣ TOP 15 NOTÍCIAS
    # =========================
    top15 = df.head(15).copy()
    top15["rank"] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x="rank",
        y="titulo",
        orientation="h",
        title="Top 15 Notícias – NIC.br"
    )
    fig1.update_layout(
        height=600,
        yaxis={"categoryorder": "array",
               "categoryarray": top15["titulo"][::-1]}
    )
    fig1.show()

    # =========================
    # 2️⃣ DISTRIBUIÇÃO POR CATEGORIA
    # =========================
    cat_count = (
        df["categoria"]
        .value_counts()
        .reset_index()
    )
    cat_count.columns = ["categoria", "quantidade"]

    fig2 = px.pie(
        cat_count,
        names="categoria",
        values="quantidade",
        title="Distribuição por Categoria – NIC.br"
    )
    fig2.show()

    # =========================
    # 3️⃣ NUVEM DE PALAVRAS
    # =========================
    texto = " ".join(df["titulo"] + " " + df["resumo"]).lower()
    palavras = re.findall(r"\b[a-záàâãéèêíïóôõöúçñ]{4,}\b", texto)

    if palavras:
        wc = (
            pd.Series(palavras)
            .value_counts()
            .head(25)
            .reset_index()
        )
        wc.columns = ["palavra", "freq"]

        fig3 = px.treemap(
            wc,
            path=["palavra"],
            values="freq",
            title="Nuvem de Palavras – NIC.br"
        )
        fig3.show()


In [18]:
# %%
plot_charts(df_filt if not df_filt.empty else df_plot)
